This will serve as the notebook for testing my topic modeling portion of this project.

After verifying that everything is working, it will be changed to a .py file to be runnable on new incoming data

In [46]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from tmtoolkit.corpus import Corpus, filter_clean_tokens, corpus_num_tokens, vocabulary_size, lemmatize

In [33]:
df = pd.read_csv(r"data/2014_funny_comments.csv")

In [34]:
df = df[~df.body.isnull()]  # Remove the missed null values

conditions = [ (df['score'] < 1), (df['score'] >= 1) & (df['score'] <= 13), (df['score'] > 13) ] 
values = ['bad', 'okay', 'good']
df['flag'] = np.select(conditions, values)  # Bin our scores into categories under the column name 'flag'
df.head()

,score,body,HOUR_int,DAY_int,MONTH_int,flag
0,1,Ain't no half steppin,0,1,1,okay
1,7,Inside the gas tank cover or on top of a tire,0,1,2,okay
2,12,Brought to you by /r/SummerReddit,0,1,7,okay
3,1,This makes me want to murder.,0,1,5,okay
4,3,I know someone who got a DUI sleeping it off i...,0,1,2,okay


In [6]:
corpus = list(df.body)  # Get full corpus of comments

In [ ]:
def preprocess(texts):
    corp = Corpus({ i:r for i, r in enumerate(texts) })
    # preproc = TMPreproc(corp, language='en')
    # preproc.pos_tag()  # If we want to filter for POS later
    preproc.lemmatize()
    preproc.tokens_to_lowercase()
    preproc.remove_special_chars_in_tokens()
    preproc.add_stopwords(['http']) # Start with just http, will add more as we go if we want to go more in-depth
    # preproc.filter_for_pos('N', 'V', 'ADJ')  # Can filter for POS if we want to later
    preproc.clean_tokens(remove_numbers = True, remove_shorter_than = 2)
    preproc.remove_common_tokens(df_threshold=0.75)  # If token in >75% of corpi, delete
    preproc.remove_uncommon_tokens(df_threshold=0.01)  # If token in <1% of corpi, delete

In [49]:
smallcorp = Corpus(corpus[:1000], language='en')  # For testing
# smallcorp

c:\Users\Zade\anaconda3\envs\py1010-red\lib\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


RuntimeError: language model "en_core_web_sm" cannot be loaded; are you sure it is installed? see https://spacy.io/models or https://tmtoolkit.readthedocs.io/en/latest/install.html for further information on installing language models

In [ ]:
pre.tokens_datatable